### Developing a best-practice approach for importing, analyzing, cleaning & visualizing

#### Loading libraries

In [2]:
from sqlalchemy import create_engine

In [17]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import shapefile as shp

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

#### Importing

In [6]:
ghg=pd.read_csv('data/caemissions.csv')
df=ghg.copy()
df.head(2)

,Year,ARBID,FACID,Facility,Primary Sector,NAICS Code,NAICS Description,Address,City,State,...,PM10,PM2.5,Benzene,13-Butadiene,Chromium Hexavalent,Diesel PM,Formaldehyde,Hydrochloric Acid,Hydrogen Sulfide,Nickel
0,2010,100001,50001051.0,ACE Cogeneration,Cogeneration,221112,Fossil Fuel Electric Power Generation,"12801 Mariposa Street, Trona, CA 93562",Trona,California,...,46.5,26.1,26.40,3.74,0.81,346.0,320.0,272430.0,NaN,14.8
1,2010,100002,131249.0,Tesoro Los Angeles Refinery - Wilmington Calciner,Other Combustion Source,324199,All Other Petroleum and Coal Products Manufact...,"1175 Carrack Avenue, Wilmington, CA 90744",Wilmington,California,...,20.1,11.3,0.03,0.00,0.32,NaN,185.0,NaN,NaN,32.4


### Fundamentals

In [9]:
# df.info()

In [8]:
df.shape

(5799, 38)

In [10]:
df.isnull().values.any()

True

In [11]:
df.isnull().sum()

Year                      0
ARBID                     0
FACID                   361
Facility                  0
Primary Sector            0
NAICS Code                0
NAICS Description         0
Address                   0
City                      0
State                     0
Zip Code                  0
Latitude                  0
Longitude                 0
Tract                     0
County                    0
District                  0
Basin                     0
Cap-and-Trade             0
Total GHG                 0
Non-Biomass GHG           0
Biomass CO2               0
CO2                       0
CH4                       0
N2O                       0
Covered GHG               0
VOC                       0
NOx                       0
SOx                       0
PM10                      0
PM2.5                     0
Benzene                1736
13-Butadiene           3754
Chromium Hexavalent    3733
Diesel PM              4125
Formaldehyde           1946
Hydrochloric Acid   

In [13]:
df["Primary Sector"].unique()

array(['Cogeneration', 'Other Combustion Source',
       'Electricity Generation', 'Cement Plant', 'Hydrogen Plant',
       'Refinery', 'Oil and Gas Production', 'Other Combustion Source '],
      dtype=object)

In [14]:
df[df.duplicated() == True]

,Year,ARBID,FACID,Facility,Primary Sector,NAICS Code,NAICS Description,Address,City,State,...,PM10,PM2.5,Benzene,13-Butadiene,Chromium Hexavalent,Diesel PM,Formaldehyde,Hydrochloric Acid,Hydrogen Sulfide,Nickel


In [19]:
df=df.rename(columns={"Total GHG ":"GHG_MT"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5799 entries, 0 to 5798
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 5799 non-null   int64  
 1   ARBID                5799 non-null   int64  
 2   FACID                5438 non-null   float64
 3   Facility             5799 non-null   object 
 4   Primary Sector       5799 non-null   object 
 5   NAICS Code           5799 non-null   int64  
 6   NAICS Description    5799 non-null   object 
 7   Address              5799 non-null   object 
 8   City                 5799 non-null   object 
 9   State                5799 non-null   object 
 10  Zip Code             5799 non-null   int64  
 11  Latitude             5799 non-null   float64
 12  Longitude            5799 non-null   float64
 13  Tract                5799 non-null   object 
 14  County               5799 non-null   object 
 15  District             5799 non-null   o

In [ ]:
df.drop(['Code'],inplace=True,axis=1)
df.head(2)

### Visualize

In [21]:
year = df.groupby('Year')['Total GHG'].sum().reset_index()
year

,Year,Total GHG
0,2010,116569418.0
1,2011,110821566.0
2,2012,124204472.0
3,2013,124179601.0
4,2014,126695197.0
5,2015,125172227.0
6,2016,113988031.0
7,2017,109621367.0
8,2018,110372154.0
9,2019,106457646.0


In [22]:
# df_World=df[df["Entity"]=="World"]
fig = go.Figure(data=go.Scatter(x=year['Year'],
                                y=year['Total GHG'],
                                mode='lines')) # hover text goes here
fig.update_layout(title='California GHG Emissions',title_x=0.5,xaxis_title="Year",yaxis_title="GHG Emissions, MT")
fig.show()

In [40]:
county = df.groupby('County')['Total GHG'].sum().reset_index()
county_sort = county.sort_values('Total GHG', ascending=False)
county_sort.head(10)

,County,Total GHG
16,Los Angeles,285138044.0
12,Kern,232668951.0
5,Contra Costa,189355600.0
29,San Bernardino,103512078.0
30,San Diego,36338198.0
41,Solano,29728888.0
36,Santa Clara,29065244.0
21,Monterey,27949777.0
27,Sacramento,25186967.0
38,Shasta,19058397.0


In [24]:
df_Kern=df[df["County"]=="Kern"]

In [26]:
df_Kern.head(2)

,Year,ARBID,FACID,Facility,Primary Sector,NAICS Code,NAICS Description,Address,City,State,...,PM10,PM2.5,Benzene,13-Butadiene,Chromium Hexavalent,Diesel PM,Formaldehyde,Hydrochloric Acid,Hydrogen Sulfide,Nickel
65,2010,100157,1119.0,Double C Limited,Cogeneration,221112,Fossil Fuel Electric Power Generation,"10245 Oilfield Road, Bakersfield, CA 93308",Bakersfield,California,...,1.1,1.1,20.80,NaN,NaN,NaN,173.0,NaN,NaN,NaN
81,2010,100189,1118.0,High Sierra Limited,Cogeneration,221112,Fossil Fuel Electric Power Generation,"10600 Oilfield Road, Bakersfield, CA 93308",Bakersfield,California,...,0.3,0.3,4.93,NaN,NaN,NaN,41.0,NaN,NaN,NaN


In [27]:
kern_sum = df_Kern.groupby('Year')['Total GHG'].sum().reset_index()
kern_sum

,Year,Total GHG
0,2010,23502572.0
1,2011,21386116.0
2,2012,23976134.0
3,2013,24905064.0
4,2014,26701219.0
5,2015,25872913.0
6,2016,23006714.0
7,2017,21797783.0
8,2018,21155779.0
9,2019,20364657.0


In [32]:
# df_test=df[df["County"]=="Kern"].groupby('Year')['Total GHG'].sum().reset_index()
# df_test

In [28]:
fig = go.Figure(data=go.Scatter(x=kern_sum['Year'],
                                y=kern_sum['Total GHG'],
                                mode='lines',
                               marker_color='darkred')) 

fig.update_layout(title='GHG Emissions in Kern County',xaxis_title="Year",yaxis_title="GHG Emissions, MT",xaxis_range=['2010','2019'])
fig.show()

In [39]:
fig = go.Figure()


kern=df[df["County"]=="Kern"].groupby('Year')['Total GHG'].sum().reset_index()
la=df[df["County"]=="Los Angeles"].groupby('Year')['Total GHG'].sum().reset_index()
sb=df[df["County"]=="San Bernardino"].groupby('Year')['Total GHG'].sum().reset_index()
sd=df[df["County"]=="San Diego"].groupby('Year')['Total GHG'].sum().reset_index()
contra=df[df["County"]=="Contra Costa"].groupby('Year')['Total GHG'].sum().reset_index()

fig.add_trace(go.Scatter(x=kern['Year'], y=kern['Total GHG'], name = 'Kern',
                         line=dict(color='royalblue', width=4,dash="dot")))

fig.add_trace(go.Scatter(x=la['Year'], y=la['Total GHG'], name = 'Los Angeles',
                         line=dict(color='green', width=4,dash="dashdot")))

fig.add_trace(go.Scatter(x=sb['Year'], y=sb['Total GHG'], name = 'San Bernardino',
                         line=dict(color='orange', width=4,dash="dash")))

fig.add_trace(go.Scatter(x=sd['Year'], y=sd['Total GHG'], name = 'San Diego',
                         line=dict(color='red', width=4,dash="dot")))

fig.add_trace(go.Scatter(x=contra['Year'], y=contra['Total GHG'], name = 'Contra Costa',
                         line=dict(color='brown', width=4,dash="dashdot")))

fig.update_layout(title='GHG Emissions Over Time For Selected CA Counties',title_x=0.5,xaxis_title="Year",yaxis_title="GHG Emissions, MT")
fig.show()

#### Helpful to QC names (i.e., duplicate entries)

In [47]:
facility = df.groupby('Facility')['Total GHG'].sum().reset_index()
facility

,Facility,Total GHG
0,3M Corona,113599.0
1,AAK Richmond,24951.0
2,AB&I Foundry,0.0
3,ABI Foundry,125738.0
4,ACE Cogeneration,2906407.0
...,...,...
807,"Wildflower Energy, LP - Larkspur Energy Facili...",321222.0
808,Windset Farms,271116.0
809,"Wm. Bolthouse Farms, Inc.",295071.0
810,"Woodland Biomass Power, Ltd",2703365.0


In [46]:
temp = df.groupby('Facility')['Total GHG'].sum().reset_index()
fig = px.treemap(temp,path = ['Facility'],values = 'Total GHG')
fig.update_layout(title='California CARB-Reported GHG Emissions by Facility',title_x=0.5)
fig.show()

In [48]:
df_2019=df[df["Year"]==2019].sort_values('Total GHG', ascending = False).reset_index()

In [57]:
df_2019v2 = df_2019[(df_2019[['Total GHG']] != 0).any(axis=1)]

In [59]:
GHG_mean=df_2019v2['Total GHG'].mean()
GHG_min=df_2019v2['Total GHG'].min()
GHG_max=df_2019v2['Total GHG'].max()
GHG_sum=df_2019v2['Total GHG'].sum()

print(GHG_mean)
print(GHG_min)
print(GHG_max)
print(GHG_sum)

186114.76573426573
3.0
5918258.0
106457646.0


In [63]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+gauge", value = GHG_mean,
    domain = {'x': [0.25, 1], 'y': [0.1, 0.2]},
    title = {'text': "Mean GHG Emissions, 2019",'font':{'color': 'black','size':15}},
    number={'font':{'color': 'black'}},
    gauge = {
        'shape': "bullet",
        'axis': {'range': [None,200000]},
        'bar': {'color': "lightblue"}}
))


fig.add_trace(go.Indicator(
    mode = "number+gauge", value = GHG_min,
    domain = {'x': [0.25, 1], 'y': [0.3, 0.4]},
    title = {'text': "Min GHG Emissions, 2019",'font':{'color': 'black','size':15}},
    number={'font':{'color': 'black'}},
    gauge = {
        'shape': "bullet",
        'axis': {'range': [None,5]},
        'bar': {'color': "cyan"}}
))

fig.add_trace(go.Indicator(
    mode = "number+gauge", value = GHG_max,
    domain = {'x': [0.25, 1], 'y': [0.5, 0.6]},
    title = {'text' :"Max GHG Emissions, 2019",'font':{'color': 'black','size':15}},
     number={'font':{'color': 'black'}},
    gauge = {
        'shape': "bullet",
        'axis': {'range': [None,10000000]},
        'bar': {'color': "darkblue"}}
))
fig.add_trace(go.Indicator(
    mode = "number+gauge", value = GHG_sum,
    domain = {'x': [0.25, 1], 'y': [0.7, 0.8]},
    title = {'text' :"Sum GHG Emissions, 2019",'font':{'color': 'black','size':15}},
     number={'font':{'color': 'black'}},
    gauge = {
        'shape': "bullet",
        'axis': {'range': [None,200000000]},
        'bar': {'color': "darkcyan"}}
))

fig.update_layout(title=" 2019 California GHG Emissions Statistics ",title_x=0.5)
fig.show()